In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [7]:
data = pd.read_csv("socialmobility.csv")
data

,father,son,count
0,farm,farm,703
1,farm,unskilled,1478
2,farm,skilled,1430
3,farm,professional,1109
4,unskilled,farm,58
5,unskilled,unskilled,1756
6,unskilled,skilled,1630
7,unskilled,professional,1568
8,skilled,farm,63
9,skilled,unskilled,1453


In [6]:
n_boots = 100
counts = data.count.values
resampler = stats.multinomial(counts.sum(), counts/counts.sum())
resampled_counts = resampler.rvs(n_boots)

# Skilled son from unskilled father

In [8]:
mask = data.father == "unskilled"
relevant = np.tile(mask, [1, n_boots]).reshape(n_boots, -1)
relevant_counts = resampled_counts[relevant].reshape([n_boots, -1])
prob_samples = relevant_counts / relevant_counts.sum(axis=1).reshape(-1, 1)

In [10]:
results = pd.DataFrame()
results["son_job"] = data[mask].son.values
results["p (MAP)"] = prob_samples.mean(axis=0)
results[["p (2.5% bound)", "p (97.5% bound)"]] = np.percentile(prob_samples, [2.5, 97.5], axis=0).T
results

,son_job,p (MAP),p (2.5% bound),p (97.5% bound)
0,farm,0.011353,0.008777,0.014361
1,unskilled,0.350463,0.339629,0.362620
2,skilled,0.324530,0.314107,0.334137
3,professional,0.313654,0.302212,0.325171


# Father occupation if son is professional

In [12]:
mask = data.son == "professional"
relevant = np.tile(mask, [1, n_boots]).reshape(n_boots, -1)
relevant_counts = resampled_counts[relevant].reshape([n_boots, -1])

prob_samples = relevant_counts / relevant_counts.sum(axis=1).reshape(-1, 1)

In [13]:
results = pd.DataFrame()
results["father_job"] = data[mask].father.values
results["p (MAP)"] = prob_samples.mean(axis=0)
results[["p (2.5% bound)", "p (97.5% bound)"]] = np.percentile(prob_samples, [2.5, 97.5], axis=0).T
results

,father_job,p (MAP),p (2.5% bound),p (97.5% bound)
0,farm,0.130205,0.123816,0.136673
1,unskilled,0.185338,0.178720,0.194144
2,skilled,0.292779,0.285267,0.301493
3,professional,0.391677,0.381828,0.400113
